План лекции (временный блок):
* GAN - Генеративно-состязательная сеть (англ. Generative adversarial network, сокращённо GAN)
 * Контент: изображения архитектуры сети
 * Контент: Генератор (архитектура)
 * Контент: Классификатор (архитектура)
* cGAN - Условные порождающие состязательные сети CGAN (Conditional Generative Adversarial Nets)
 * Картинка принципа работы (в сеть подаются так же изображения)
 * like above: арихитектура cGAN
* DCGAN (Deep Convolutional Generative Adversarial Nets)
 * де-факто это cGAN, только для изображений, картинки похожи
* StackGAN - порождающая состязательная сеть для генерации фото-реалистичных изображений исходя из текстового описания
* LAPGAN (Laplacian Pyramid of Adversarial Networks)
* ControlGAN (Controllable Generative Adversarial Networks)
* AC-GAN Auxiliary Classifier Generative Adversarial Networks

## Ниже - лекция Техносферы

https://www.youtube.com/watch?v=hPux6TVtM58%D0%BF

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-03.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-06.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-08.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-09.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-10.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-11.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-13.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-15.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-16.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-17.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-18.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-19.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-20.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-22.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-23.gif" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-24.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-26.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-27.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-28.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-29.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-30.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-31.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-32.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-34.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-35.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-36.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-37.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-38.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-40.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-41.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-42.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-43.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-44.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-45.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-46.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-47.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-48.png" width="700">